In [1]:
import pandas as pd
import rdkit
from rdkit import Chem
from standardiser import standardise
import logging
from rdkit.Chem import Descriptors

In [2]:
METAL_ELEMENTS = ['Li', 'Be', 'Na', 'Mg', 'Al', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co',
                  'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh',
                  'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Cs', 'Ba', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os',
                  'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'Fr', 'Ra', 'Lr', 'Ho']

In [3]:
# 加载csv数据
df = pd.read_csv('hERG.csv')

In [4]:
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL1095877,NaN,None,377.92,0,2.54,11m,O=C([C@H]1C[C@H](Oc2cccc(Cl)c2)CN1)N1CCCN(C2CC...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1158560,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
1,CHEMBL1090373,NaN,None,537.58,1,3.51,16,O=C(c1cnccn1)N1CCN(CCCNc2nnc(-c3cccc([N+](=O)[...,Inhibition,NaN,...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1156300,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
2,CHEMBL1094521,NaN,None,505.83,2,6.48,3,C[C@H](O)C(=O)NC1CC(C)(C)Oc2nc(-c3ccc(Cl)cc3Cl...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1157794,1.0,Scientific Literature,J Med Chem,2010.0,CHEMBL3307715,NaN
3,CHEMBL1169572,NaN,None,539.38,2,6.67,8c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CHEMBL358075,NaN,None,383.44,0,3.58,1b,O=S(=O)(c1ccc(F)cc1)C1CCN(CCc2ccc(F)cc2F)CC1,Ki,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1142766,1.0,Scientific Literature,Bioorg Med Chem Lett,2005.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26449,CHEMBL4857929,NaN,None,392.39,0,2.71,3i,CNc1nc(Nc2cnc(C#N)c(N[C@@H]3CCCNC3)c2)ncc1C(F)...,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4828771,1.0,Scientific Literature,J Med Chem,2021.0,None,Dose = 10.0 microM
26450,CHEMBL4876703,NaN,None,513.54,1,2.91,120,COc1ncc2cc1NS(=O)(=O)c1cccc(c1)C(=O)NCc1cncc(c...,IC50,'>',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4819017,1.0,Scientific Literature,Eur J Med Chem,2021.0,None,NaN
26451,CHEMBL4861097,NaN,None,344.38,0,3.11,12,O=C(NCc1ccncc1)Nc1ccc2nc(-c3ccccc3)nn2c1,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4819093,1.0,Scientific Literature,Bioorg Med Chem Lett,2021.0,None,Dose = 30.0 microM
26452,CHEMBL5077755,NaN,None,471.38,1,5,5,O=S(=O)(CC1CCN(CCCc2noc3cc(F)ccc23)C1)c1ccc(Cl...,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL5046222,1.0,Scientific Literature,J Med Chem,2021.0,None,Dose = 1.0 microM


In [5]:
# 删除Smiles为空值的行
df.dropna(axis=0, subset=["Smiles"], inplace=True)

In [6]:
#去金属离子
# df['Smiles'].str.contains(metal)取出包含这个离子的
for metal in METAL_ELEMENTS:
    df = df[~df['Smiles'].str.contains(metal)]
    print("no {}".format(metal),df.shape)    

no Li (24839, 45)
no Be (24839, 45)
no Na (24823, 45)
no Mg (24823, 45)
no Al (24823, 45)
no K (24821, 45)
no Ca (24820, 45)
no Sc (24413, 45)
no Ti (24413, 45)
no V (24413, 45)
no Cr (24413, 45)
no Mn (24413, 45)
no Fe (24413, 45)
no Co (24413, 45)
no Ni (24413, 45)
no Cu (24413, 45)
no Zn (24409, 45)
no Ga (24409, 45)
no Ge (24409, 45)
no Rb (24409, 45)
no Sr (24407, 45)
no Y (24407, 45)
no Zr (24407, 45)
no Nb (24407, 45)
no Mo (24407, 45)
no Tc (24407, 45)
no Ru (24407, 45)
no Rh (24407, 45)
no Pd (24407, 45)
no Ag (24407, 45)
no Cd (24407, 45)
no In (24407, 45)
no Sn (24407, 45)
no Sb (24407, 45)
no Cs (24407, 45)
no Ba (24407, 45)
no Lu (24407, 45)
no Hf (24407, 45)
no Ta (24407, 45)
no W (24407, 45)
no Re (24407, 45)
no Os (24407, 45)
no Ir (24407, 45)
no Pt (24407, 45)
no Au (24407, 45)
no Hg (24407, 45)
no Tl (24407, 45)
no Pb (24407, 45)
no Bi (24407, 45)
no Po (24407, 45)
no Fr (24407, 45)
no Ra (24407, 45)
no Lr (24407, 45)
no Ho (24407, 45)


In [7]:
# 分子标准化
for i in df.index:
    try:
        smi = df.loc[i, 'Smiles']
        # print(smi)
        mol = Chem.MolFromSmiles(smi)
        mol = Chem.AddHs(mol)
        parent = standardise.run(mol)
        mol_ok_smi = Chem.MolToSmiles(parent)
        df.loc[i, 'Smiles'] = mol_ok_smi
        # print(i, 'done')
    except standardise.StandardiseException as e:
        logging.warning(e.message)

[10:02:21] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:28] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:30] Can't kekulize mol.  Unkekulized atoms: 1
[10:02:31] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:31] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:33] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:33] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:35] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:40] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:44] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:44] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:45] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:45] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:02:47] Explicit valence for atom # 0 O, 3, is greater than permitte

In [8]:
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL1095877,NaN,None,377.92,0,2.54,11m,[H]c1c([H])c(Cl)c([H])c(O[C@]2([H])C([H])([H])...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1158560,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
1,CHEMBL1090373,NaN,None,537.58,1,3.51,16,[H]c1nc([H])c(C(=O)N2C([H])([H])C([H])([H])N(C...,Inhibition,NaN,...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1156300,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
2,CHEMBL1094521,NaN,None,505.83,2,6.48,3,[H]O[C@]([H])(C(=O)N([H])C1([H])c2c(nc(-c3c([H...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1157794,1.0,Scientific Literature,J Med Chem,2010.0,CHEMBL3307715,NaN
4,CHEMBL358075,NaN,None,383.44,0,3.58,1b,[H]c1c([H])c(S(=O)(=O)C2([H])C([H])([H])C([H])...,Ki,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1142766,1.0,Scientific Literature,Bioorg Med Chem Lett,2005.0,None,NaN
5,CHEMBL1809061,NaN,None,435.92,0,3.61,18,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1806498,1.0,Scientific Literature,Bioorg Med Chem Lett,2011.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26449,CHEMBL4857929,NaN,None,392.39,0,2.71,3i,[H]c1nc(C#N)c(N([H])[C@@]2([H])C([H])([H])N([H...,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4828771,1.0,Scientific Literature,J Med Chem,2021.0,None,Dose = 10.0 microM
26450,CHEMBL4876703,NaN,None,513.54,1,2.91,120,[H]c1nc(OC([H])([H])[H])c2c([H])c1-c1nn3c(c([H...,IC50,'>',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4819017,1.0,Scientific Literature,Eur J Med Chem,2021.0,None,NaN
26451,CHEMBL4861097,NaN,None,344.38,0,3.11,12,[H]c1nc([H])c([H])c(C([H])([H])N([H])C(=O)N([H...,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL4819093,1.0,Scientific Literature,Bioorg Med Chem Lett,2021.0,None,Dose = 30.0 microM
26452,CHEMBL5077755,NaN,None,471.38,1,5,5,[H]c1c([H])c(S(=O)(=O)C([H])([H])C2([H])C([H])...,Inhibition,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL5046222,1.0,Scientific Literature,J Med Chem,2021.0,None,Dose = 1.0 microM


In [9]:
#删除重复值
df.drop_duplicates(keep='first', inplace=True)
print('删除重复值df:', df.shape)

删除重复值df: (24377, 45)


In [10]:
#删除Standard type中非'IC50 EC50 Ki Kd'的数据
#提取Standard type为IC50
df1=df[df['Standard Type'].isin(["IC50"])]

In [11]:
#df2=df[df['Standard Type'].isin(["EC50"])]

In [12]:
#提取Standard type为Ki
df3=df[df['Standard Type'].isin(["Ki"])]

In [13]:
#提取Standard type为Kd
df4=df[df['Standard Type'].isin(["Kd"])]

In [14]:
# result = df1.append(df2).append(df3).append(df4)
# frames = [df1, df2, df3, df4]
frames = [df1, df3, df4]
df = pd.concat(frames)
# print('删除Standard type中非IC50 EC50 Ki Kd的数据:', df.shape)
#print('删除Standard type中非IC50 Ki Kd的数据:', df.shape)

In [15]:
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL1095877,NaN,None,377.92,0,2.54,11m,[H]c1c([H])c(Cl)c([H])c(O[C@]2([H])C([H])([H])...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1158560,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
2,CHEMBL1094521,NaN,None,505.83,2,6.48,3,[H]O[C@]([H])(C(=O)N([H])C1([H])c2c(nc(-c3c([H...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1157794,1.0,Scientific Literature,J Med Chem,2010.0,CHEMBL3307715,NaN
5,CHEMBL1809061,NaN,None,435.92,0,3.61,18,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1806498,1.0,Scientific Literature,Bioorg Med Chem Lett,2011.0,None,NaN
7,CHEMBL1098752,NaN,None,451.46,0,2.05,24,[H]c1nc([H])c([H])c(N([H])c2nc(N3C([H])([H])C(...,IC50,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1157119,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN
8,CHEMBL1802915,NaN,None,495.56,0,2.15,80,[H]c1nc2c([H])c([H])c3c([H])c([H])c(N([H])S(=O...,IC50,'>',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL1800066,1.0,Scientific Literature,J Med Chem,2011.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,CHEMBL3590496,NaN,None,655.69,2,6.66,41,[H]c1c([H])c(OC([H])([H])[H])c([H])c(-c2c([H])...,Kd,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL3588758,1.0,Scientific Literature,J Med Chem,2015.0,CHEMBL3307715,NaN
21279,CHEMBL3590449,NaN,None,389.36,0,2.51,6,[H]OC(=O)C([H])([H])N(C([H])([H])C([H])([H])Oc...,Kd,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL3588758,1.0,Scientific Literature,J Med Chem,2015.0,CHEMBL3307715,NaN
22225,CHEMBL3590445,NaN,None,345.36,0,3.06,2,[H]c1c([H])c([N+](=O)[O-])c([H])c([H])c1OC([H]...,Kd,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL3588758,1.0,Scientific Literature,J Med Chem,2015.0,CHEMBL3307715,NaN
23606,CHEMBL3590468,NaN,None,571.79,1,8.28,30,[H]c1c([H])c(C#CC([H])([H])C([H])([H])C([H])([...,Kd,'=',...,HERG,Homo sapiens,SINGLE PROTEIN,CHEMBL3588758,1.0,Scientific Literature,J Med Chem,2015.0,CHEMBL3307715,NaN


In [16]:
#删除Standard Relation中非'='的数据
df=df[df['Standard Relation'].isin(["'='"])]
print('删除非=的df:', df.shape)

删除非=的df: (9635, 45)


In [17]:
#删除assay_type中非"B"的数据B=binding,F=functional
df=df[df['Assay Type'].isin(["B"])]
print('删除非B的df:', df.shape)

删除非B的df: (8912, 45)


In [18]:
#删除三个subset列中有空值的行
df=df.dropna(how='any',subset=(['Smiles','Standard Units','Molecule ChEMBL ID','Standard Value']))
print('删除存在空值的df:',df.shape)

删除存在空值的df: (8912, 45)


In [19]:
# 计算分子MW
molweight = []
for smi in list(df['Smiles']):
    molweight.append(Descriptors.MolWt(Chem.MolFromSmiles(smi)))
df['molecular_weight'] = molweight

In [20]:
# 删除分子量大于1000的
df = df[ df['molecular_weight']<=1000 ]

In [21]:
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,molecular_weight
0,CHEMBL1095877,NaN,None,377.92,0,2.54,11m,[H]c1c([H])c(Cl)c([H])c(O[C@]2([H])C([H])([H])...,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1158560,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN,377.916
2,CHEMBL1094521,NaN,None,505.83,2,6.48,3,[H]O[C@]([H])(C(=O)N([H])C1([H])c2c(nc(-c3c([H...,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1157794,1.0,Scientific Literature,J Med Chem,2010.0,CHEMBL3307715,NaN,505.829
5,CHEMBL1809061,NaN,None,435.92,0,3.61,18,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1806498,1.0,Scientific Literature,Bioorg Med Chem Lett,2011.0,None,NaN,435.919
7,CHEMBL1098752,NaN,None,451.46,0,2.05,24,[H]c1nc([H])c([H])c(N([H])c2nc(N3C([H])([H])C(...,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1157119,1.0,Scientific Literature,Bioorg Med Chem Lett,2010.0,None,NaN,451.457
10,CHEMBL1085396,NaN,None,431.97,1,6.55,liu74,[H]c1c([H])c(-c2noc(C([H])([H])[H])c2-c2nc3c([...,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1821550,1.0,Scientific Literature,Bioorg Med Chem Lett,2011.0,None,NaN,431.967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26114,CHEMBL2376799,NaN,None,272.36,0,0.92,32,[H]c1nc2c(N3C([H])([H])C([H])([H])N(C([H])([H]...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL2375287,1.0,Scientific Literature,Bioorg Med Chem Lett,2013.0,None,NaN,272.356
26115,CHEMBL2376805,NaN,None,272.36,0,1.14,60,[H]c1nc2c(N3C([H])([H])C([H])(N([H])C([H])([H]...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL2375287,1.0,Scientific Literature,Bioorg Med Chem Lett,2013.0,None,NaN,272.356
26211,CHEMBL5088467,NaN,None,444.51,0,3.95,1,[H]c1nc(N(C(=O)c2c([H])c([H])c(-c3c([H])nn4c([...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5034046,1.0,Scientific Literature,ACS Med Chem Lett,2022.0,None,NaN,444.514
26290,CHEMBL278558,NaN,None,365.42,0,2.2,25,[H]c1c([H])c(C(=O)N([H])[C@]2([H])C([H])([H])C...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1147603,1.0,Scientific Literature,Bioorg Med Chem Lett,2004.0,None,NaN,365.424


In [22]:
# 计算分子logP
logP = []
for smi in list(df['Smiles']):
    logP.append(Descriptors.MolLogP(Chem.MolFromSmiles(smi)))
df['logP'] = logP

In [ ]:
#单位换算
df['molecular_weight']=df['molecular_weight'].astype("float64")

In [ ]:
df2 = df[df['Standard Units'].isin(["nM"])]
print('单位为nM的df:', df2.shape)

单位为nM的df: (8895, 47)


In [ ]:
df3 = df[df['Standard Units'].isin(["ug.mL-1"])]
print('单位为ug/ml的df:', df3.shape)

单位为ug/ml的df: (8, 47)


In [ ]:
df.shape

(8903, 47)

In [ ]:
#将ug/ml换算为nM
df3['Standard Value'] = df3['Standard Value']/df3['molecular_weight']*1000000    #g*e-6/L*e-3  /  (g/mol)   = e-9*mol/L   Molecular Weight是M分子质量
df3['Standard Units'] = "nM"
#df4['translate'] = df4['Standard Value']*1000
#df4['Standard Value'] = df4['translate']
df = df2.append(df3)

/root/anaconda3/envs/aidd/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/root/anaconda3/envs/aidd/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#此时所有单位都换算为nM
df.shape

(8903, 47)

In [ ]:
#df = df2.append(df4)
print('单位换算后的df:',df.shape)
# df.to_csv('test_{}.csv'.format(name),index=None)

单位换算后的df: (8903, 47)


In [ ]:
#根据相同smiles的取Standard Value的平均值
df_mean = df.groupby('Molecule ChEMBL ID')['Standard Value'].mean()
print('df',df_mean.shape)

df (7834,)


In [ ]:
df_mean

Molecule ChEMBL ID
CHEMBL1000       56133.013333
CHEMBL100454     11481.540000
CHEMBL1008         549.702500
CHEMBL1077638     4120.000000
CHEMBL1077647    19800.000000
                     ...     
CHEMBL95         14312.000000
CHEMBL95463      10000.000000
CHEMBL96153        177.460000
CHEMBL98           322.000000
CHEMBL998        11965.440000
Name: Standard Value, Length: 7834, dtype: float64

In [ ]:
#将df_mean转换成表
# df_mean = pd.DataFrame({'standard_value':df_mean})
df_mean_dict = df_mean.to_dict()

In [ ]:
df_mean_dict

{'CHEMBL1000': 56133.013333333336,
 'CHEMBL100454': 11481.54,
 'CHEMBL1008': 549.7025,
 'CHEMBL1077638': 4120.0,
 'CHEMBL1077647': 19800.0,
 'CHEMBL1078162': 52400.0,
 'CHEMBL1078169': 12900.0,
 'CHEMBL1078231': 11900.0,
 'CHEMBL1078742': 17200.0,
 'CHEMBL1078983': 8300.0,
 'CHEMBL1079264': 7920.0,
 'CHEMBL1079313': 180.0,
 'CHEMBL1079461': 1584.89,
 'CHEMBL1079475': 3162.28,
 'CHEMBL1079480': 794.33,
 'CHEMBL1079481': 1000.0,
 'CHEMBL1079497': 125.89,
 'CHEMBL1079577': 630.96,
 'CHEMBL1079639': 199.53,
 'CHEMBL1079640': 794.33,
 'CHEMBL1079667': 1584.89,
 'CHEMBL1079822': 630.96,
 'CHEMBL1079823': 398.11,
 'CHEMBL1079824': 1000.0,
 'CHEMBL1079938': 1000.0,
 'CHEMBL1079980': 1995.26,
 'CHEMBL1080046': 2511.89,
 'CHEMBL1080117': 1097.395,
 'CHEMBL1080189': 6309.57,
 'CHEMBL1080293': 158.49,
 'CHEMBL1080485': 4200.0,
 'CHEMBL1080490': 794.33,
 'CHEMBL1080584': 430.0,
 'CHEMBL1080789': 7140.0,
 'CHEMBL1080790': 16800.0,
 'CHEMBL1080846': 1000.0,
 'CHEMBL1080968': 10300.0,
 'CHEMBL1080969'

In [ ]:
# print(df_mean)
df['standard_value_mean'] = df['Molecule ChEMBL ID'].apply(lambda x:df_mean_dict[x])#添加 每个化合物的平均值

In [ ]:
df.shape

(8903, 48)

In [ ]:
df.drop('Standard Value', axis=1, inplace=True)  #未平均的值已经求完，可舍弃

In [ ]:
df.drop_duplicates(subset=['Molecule ChEMBL ID'],inplace=True)   #对BDB_ID去重，因为现在有很多求过平均没删掉的数
df.reset_index(inplace=True)
print('取平均值后的df:',df.shape)

取平均值后的df: (7834, 48)


In [ ]:
# 标签转换
df.loc[df['standard_value_mean']<=10000,'standard_value_mean']=1
df.loc[df['standard_value_mean']>10000,'standard_value_mean']=0
print('转换平均值后的df：',df.shape)
df = pd.DataFrame(df,columns = ['Smiles','standard_value_mean'])
# df = pd.DataFrame(df,columns = ['Smiles','standard_value_mean'，'molecular_weight'，'logp'])
# 修改列名
df.columns = ['SMILES','LABEL']

转换平均值后的df： (7834, 48)


In [ ]:
# 保存清洗后的数据
df.to_csv('hERG_data_washed.csv', index=None)